In [1]:
import base64
import collections
import copy
import cProfile
import datetime
import gc
import itertools
import json
import math
import os
import operator
import pickle
import random
import re
import shutil
import sys
import time

import bokeh
import cv2
import hyperopt
from hyperopt import hp
import Image
import keras
from keras import *
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL as pil
import prophet
import pyflux
import pylab
import scipy
from scipy import signal
import seaborn as sns
import skimage
import sklearn
from sklearn import *
import statsmodels as sm
import tensorflow as tf
import tqdm

np.random.seed(1337)

%matplotlib inline

sns.set(font_scale=1.3)
mpl.rcParams['figure.figsize'] = 20, 12
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

linewidth = 1.0
dotsize = 15

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/lo

In [2]:
train_raw = pd.read_json('/media/ntfs/data/iceberg_classification/input/train.json')
train_size = len(train_raw)
print train_size

train_x = np.zeros((train_size, 75, 75, 2))
train_y = np.zeros(train_size)

for i in range(train_size):
    for channel in range(2):
        train_x[i, :, :, channel] = (np.asarray(train_raw.iloc[i, channel]).reshape(75, 75) + 50) / 50
    train_y[i] = train_raw.iloc[i, 4]
    
del train_raw
gc.collect()

1604


56

In [3]:
test_raw = pd.read_json('/media/ntfs/data/iceberg_classification/input/test.json')
test_size = len(test_raw)
print test_size

test_x = np.zeros((test_size, 75, 75, 2))

for i in range(test_size):
    for channel in range(2):
        test_x[i, :, :, channel] = (np.asarray(test_raw.iloc[i, channel]).reshape(75, 75) + 50) / 50
        
test_ids = test_raw[['id']].copy()

del test_raw
gc.collect()

8424


67

In [5]:
def create_model(
        kernel_size,
    
        n_filters_input,
        activation_input,
        max_pooling_size_input,
        dropout_input,
    
        n_conv_layers,
        n_filters_conv,
        activation_conv,
        max_pooling_size_conv,
        dropout_conv,
    
        n_dense_layers,
        n_dense_neurons,
        activation_dense,
        dropout_dense,
    
        optimizer,
        loss):
    
    model = models.Sequential()

    # input layer
    model.add(layers.Conv2D(
                n_filters_input,
                (kernel_size, kernel_size),
                activation=activation_input,
                input_shape=(75, 75, 2)))
    model.add(layers.MaxPooling2D((max_pooling_size_input, max_pooling_size_input), dim_ordering='th'))
    model.add(keras.layers.Dropout(dropout_input))
    
    # conv layers
    for i in range(n_conv_layers):
        model.add(layers.Conv2D(
                    n_filters_conv,
                    (kernel_size, kernel_size),
                    activation=activation_conv))
        model.add(layers.MaxPooling2D((max_pooling_size_conv, max_pooling_size_conv), dim_ordering='th'))
        model.add(keras.layers.Dropout(dropout_conv))

    model.add(keras.layers.Flatten())
        
    # dense layers
    for i in range(n_dense_layers):
        model.add(keras.layers.Dense(n_dense_neurons, activation=activation_dense))
        model.add(keras.layers.Dropout(dropout_dense))
    
    #sigmoid layer
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

In [6]:
model = create_model(
            kernel_size=3,
    
            n_filters_input=16,
            activation_input='tanh',
            max_pooling_size_input=2,
            dropout_input=0.3,

            n_conv_layers=2,
            n_filters_conv=128,
            activation_conv='relu',
            max_pooling_size_conv=2,
            dropout_conv=0.6,

            n_dense_layers=2,
            n_dense_neurons=128,
            activation_dense='relu',
            dropout_dense=0.3,

            optimizer='rmsprop',
            loss='binary_crossentropy')

In [7]:
model.fit(
        x=train_x,
        y=train_y,
        epochs=60,
        batch_size=16,
        verbose=0)

In [9]:
preds = model.predict(test_x)

In [19]:
sub = pd.DataFrame({'id': test_ids.id, 'is_iceberg': preds[:, 0]}, index=range(len(test_ids)))
sub.head(20)

,id,is_iceberg
0,5941774d,2.645342e-02
1,4023181e,4.945017e-01
2,b20200e4,9.744549e-03
3,e7f018bb,9.961261e-01
4,4371c8c3,4.398057e-01
5,a8d9b1fd,3.227885e-03
6,29e7727e,1.560408e-01
7,92a51ffb,9.998199e-01
8,c769ac97,5.312380e-07
9,aee0547d,8.188709e-08


In [21]:
sub.to_csv('/media/ntfs/data/iceberg_classification/output/sub_cnn_hyperopt_arch_001', index=False)